In [1]:
import pandas as pd
import numpy as np
import prophet as pt
import datetime as dt
import os
import matplotlib.pyplot as plt
from prophet.plot import add_changepoints_to_plot

Importing plotly failed. Interactive plots will not work.


In [9]:
files  = os.listdir("../full_data")
columns = ["grid_square", "time", "cc", "sms_in", "sms_out", "call_in", "call_out", "internet"]

In [10]:
def convert(t):
    """
    Function to convert the time steops to a standard form add 
    subtracting the hour to be in the GMT time.
    """
    return dt.datetime.fromtimestamp(t / 1e3) - dt.timedelta(hours = 1)

In [11]:
files

['.ipynb_checkpoints',
 'sms-call-internet-tn-2013-11-01.txt',
 'sms-call-internet-tn-2013-11-02.txt',
 'sms-call-internet-tn-2013-11-03.txt',
 'sms-call-internet-tn-2013-11-04.txt',
 'sms-call-internet-tn-2013-11-05.txt',
 'sms-call-internet-tn-2013-11-06.txt',
 'sms-call-internet-tn-2013-11-07.txt',
 'sms-call-internet-tn-2013-11-08.txt',
 'sms-call-internet-tn-2013-11-09.txt',
 'sms-call-internet-tn-2013-11-10.txt',
 'sms-call-internet-tn-2013-11-11.txt',
 'sms-call-internet-tn-2013-11-12.txt',
 'sms-call-internet-tn-2013-11-13.txt',
 'sms-call-internet-tn-2013-11-14.txt',
 'sms-call-internet-tn-2013-11-15.txt',
 'sms-call-internet-tn-2013-11-16.txt',
 'sms-call-internet-tn-2013-11-17.txt',
 'sms-call-internet-tn-2013-11-18.txt',
 'sms-call-internet-tn-2013-11-19.txt',
 'sms-call-internet-tn-2013-11-20.txt',
 'sms-call-internet-tn-2013-11-21.txt',
 'sms-call-internet-tn-2013-11-22.txt',
 'sms-call-internet-tn-2013-11-23.txt',
 'sms-call-internet-tn-2013-11-24.txt',
 'sms-call-intern

In [46]:
#Iterate over all files
full_data = pd.DataFrame()
for f in files[1:]:
    #get day number
    day = f[-9:-4]
    #Read the file
    df = pd.read_csv("../full_data/{}".format(f),sep="\t", header=None, names= columns)    
    #convert time column to a readable time
    df["time"] = df["time"].apply(convert)
    df["time"] = pd.to_datetime(df["time"])
    #select italy dataframe
    df = df[df["cc"] == 39]
    #grouping the data (each time step we have a value for each grid)
    df = df.groupby(['time','grid_square']).sum()
    #define a new aggregated column
    df["internet_cdr"] = df.loc[:, ["internet"]]
    #drop the unwanted columns
    df.drop(columns = ["cc","sms_in", "sms_out", "call_in", "call_out", "internet"], inplace = True)
    #Concate df with full data
    full_data = pd.concat([full_data, df])
    #save the data to a csv
    #Check point
    print('===========> Day {} is done!'.format(day))
    print('===========> Full data rows: {} rows'.format(len(full_data)))

full_data.to_csv('full_grid.csv')
print('===========> Full data size: {} Mb'.format(os.path.getsize('full_grid.csv')/10**6))

===========> Day 11-01 is done!
===========> Full data rows: 899111 rows
===========> Day 11-02 is done!
===========> Full data rows: 1798320 rows
===========> Day 11-03 is done!
===========> Full data rows: 2696916 rows
===========> Day 11-04 is done!
===========> Full data rows: 3595743 rows
===========> Day 11-05 is done!
===========> Full data rows: 4494546 rows
===========> Day 11-06 is done!
===========> Full data rows: 5393776 rows
===========> Day 11-07 is done!
===========> Full data rows: 6292907 rows
===========> Day 11-08 is done!
===========> Full data rows: 7191418 rows
===========> Day 11-09 is done!
===========> Full data rows: 8090041 rows
===========> Day 11-10 is done!
===========> Full data rows: 8988461 rows
===========> Day 11-11 is done!
===========> Full data rows: 9888005 rows
===========> Day 11-12 is done!
===========> Full data rows: 10786992 rows
===========> Day 11-13 is done!
===========> Full data rows: 11685924 rows
===========> Day 11-14 is done!
=====

this data has some null values because the desired number of rows must equal = 
`144(samples perday)*6259(grids)*62(days) = 55 880 652`

# Resampling Data every 10 mins:

In [2]:
df = pd.read_csv("full_grid.csv")

In [3]:
df = df.reset_index()
df["time"] = pd.to_datetime(df["time"])
df = df.set_index("time")
df = df.groupby("grid_square")
df_groups = list(df.groups.keys())

In [4]:
final_df = pd.DataFrame()
for grid in df_groups:
    df_g = df.get_group(grid)
    df_g = df_g.resample("10T", origin="2013-11-1 00:00:00").sum()
    final_df = pd.concat([final_df,df_g])
    print("============= grid {} resampled =====".format(grid))
final_df.drop(columns = ["index"], inplace = True)
final_df.to_csv("resampled_full_grid.csv", index = True)

============= grid 38 resampled =====
============= grid 39 resampled =====
============= grid 40 resampled =====
============= grid 154 resampled =====
============= grid 155 resampled =====
============= grid 156 resampled =====
============= grid 157 resampled =====
============= grid 158 resampled =====
============= grid 159 resampled =====
============= grid 160 resampled =====
============= grid 271 resampled =====
============= grid 272 resampled =====
============= grid 273 resampled =====
============= grid 274 resampled =====
============= grid 275 resampled =====
============= grid 276 resampled =====
============= grid 277 resampled =====
============= grid 278 resampled =====
============= grid 285 resampled =====
============= grid 286 resampled =====
============= grid 288 resampled =====
============= grid 289 resampled =====
============= grid 388 resampled =====
============= grid 389 resampled =====
============= grid 390 resampled =====
============= grid 391 resam

============= grid 1297 resampled =====
============= grid 1321 resampled =====
============= grid 1322 resampled =====
============= grid 1323 resampled =====
============= grid 1324 resampled =====
============= grid 1325 resampled =====
============= grid 1326 resampled =====
============= grid 1327 resampled =====
============= grid 1328 resampled =====
============= grid 1329 resampled =====
============= grid 1330 resampled =====
============= grid 1331 resampled =====
============= grid 1332 resampled =====
============= grid 1333 resampled =====
============= grid 1334 resampled =====
============= grid 1335 resampled =====
============= grid 1336 resampled =====
============= grid 1337 resampled =====
============= grid 1338 resampled =====
============= grid 1339 resampled =====
============= grid 1340 resampled =====
============= grid 1341 resampled =====
============= grid 1342 resampled =====
============= grid 1343 resampled =====
============= grid 1344 resampled =====


============= grid 2001 resampled =====
============= grid 2002 resampled =====
============= grid 2003 resampled =====
============= grid 2004 resampled =====
============= grid 2005 resampled =====
============= grid 2006 resampled =====
============= grid 2007 resampled =====
============= grid 2008 resampled =====
============= grid 2009 resampled =====
============= grid 2010 resampled =====
============= grid 2011 resampled =====
============= grid 2013 resampled =====
============= grid 2014 resampled =====
============= grid 2016 resampled =====
============= grid 2017 resampled =====
============= grid 2018 resampled =====
============= grid 2020 resampled =====
============= grid 2021 resampled =====
============= grid 2022 resampled =====
============= grid 2023 resampled =====
============= grid 2024 resampled =====
============= grid 2025 resampled =====
============= grid 2026 resampled =====
============= grid 2027 resampled =====
============= grid 2028 resampled =====


============= grid 2391 resampled =====
============= grid 2392 resampled =====
============= grid 2393 resampled =====
============= grid 2394 resampled =====
============= grid 2395 resampled =====
============= grid 2396 resampled =====
============= grid 2397 resampled =====
============= grid 2398 resampled =====
============= grid 2400 resampled =====
============= grid 2401 resampled =====
============= grid 2402 resampled =====
============= grid 2462 resampled =====
============= grid 2463 resampled =====
============= grid 2464 resampled =====
============= grid 2465 resampled =====
============= grid 2466 resampled =====
============= grid 2467 resampled =====
============= grid 2468 resampled =====
============= grid 2469 resampled =====
============= grid 2470 resampled =====
============= grid 2471 resampled =====
============= grid 2472 resampled =====
============= grid 2473 resampled =====
============= grid 2474 resampled =====
============= grid 2475 resampled =====


============= grid 2830 resampled =====
============= grid 2831 resampled =====
============= grid 2832 resampled =====
============= grid 2833 resampled =====
============= grid 2834 resampled =====
============= grid 2835 resampled =====
============= grid 2836 resampled =====
============= grid 2837 resampled =====
============= grid 2838 resampled =====
============= grid 2839 resampled =====
============= grid 2840 resampled =====
============= grid 2841 resampled =====
============= grid 2842 resampled =====
============= grid 2843 resampled =====
============= grid 2844 resampled =====
============= grid 2845 resampled =====
============= grid 2846 resampled =====
============= grid 2847 resampled =====
============= grid 2848 resampled =====
============= grid 2849 resampled =====
============= grid 2850 resampled =====
============= grid 2851 resampled =====
============= grid 2852 resampled =====
============= grid 2853 resampled =====
============= grid 2854 resampled =====


...

============= grid 9021 resampled =====
============= grid 9022 resampled =====
============= grid 9023 resampled =====
============= grid 9024 resampled =====
============= grid 9025 resampled =====
============= grid 9026 resampled =====
============= grid 9027 resampled =====
============= grid 9028 resampled =====
============= grid 9029 resampled =====
============= grid 9030 resampled =====
============= grid 9031 resampled =====
============= grid 9036 resampled =====
============= grid 9037 resampled =====
============= grid 9038 resampled =====
============= grid 9039 resampled =====
============= grid 9040 resampled =====
============= grid 9041 resampled =====
============= grid 9042 resampled =====
============= grid 9043 resampled =====
============= grid 9044 resampled =====
============= grid 9045 resampled =====
============= grid 9046 resampled =====
============= grid 9047 resampled =====
============= grid 9048 resampled =====
============= grid 9049 resampled =====


============= grid 9380 resampled =====
============= grid 9381 resampled =====
============= grid 9382 resampled =====
============= grid 9383 resampled =====
============= grid 9384 resampled =====
============= grid 9385 resampled =====
============= grid 9387 resampled =====
============= grid 9388 resampled =====
============= grid 9389 resampled =====
============= grid 9390 resampled =====
============= grid 9391 resampled =====
============= grid 9392 resampled =====
============= grid 9393 resampled =====
============= grid 9394 resampled =====
============= grid 9395 resampled =====
============= grid 9396 resampled =====
============= grid 9397 resampled =====
============= grid 9398 resampled =====
============= grid 9399 resampled =====
============= grid 9400 resampled =====
============= grid 9401 resampled =====
============= grid 9402 resampled =====
============= grid 9403 resampled =====
============= grid 9404 resampled =====
============= grid 9405 resampled =====


============= grid 9748 resampled =====
============= grid 9750 resampled =====
============= grid 9751 resampled =====
============= grid 9752 resampled =====
============= grid 9753 resampled =====
============= grid 9754 resampled =====
============= grid 9755 resampled =====
============= grid 9756 resampled =====
============= grid 9757 resampled =====
============= grid 9758 resampled =====
============= grid 9759 resampled =====
============= grid 9760 resampled =====
============= grid 9761 resampled =====
============= grid 9762 resampled =====
============= grid 9763 resampled =====
============= grid 9764 resampled =====
============= grid 9765 resampled =====
============= grid 9766 resampled =====
============= grid 9767 resampled =====
============= grid 9768 resampled =====
============= grid 9769 resampled =====
============= grid 9800 resampled =====
============= grid 9801 resampled =====
============= grid 9802 resampled =====
============= grid 9803 resampled =====


============= grid 10156 resampled =====
============= grid 10157 resampled =====
============= grid 10158 resampled =====
============= grid 10159 resampled =====
============= grid 10160 resampled =====
============= grid 10161 resampled =====
============= grid 10162 resampled =====
============= grid 10163 resampled =====
============= grid 10164 resampled =====
============= grid 10165 resampled =====
============= grid 10166 resampled =====
============= grid 10167 resampled =====
============= grid 10168 resampled =====
============= grid 10169 resampled =====
============= grid 10170 resampled =====
============= grid 10171 resampled =====
============= grid 10198 resampled =====
============= grid 10199 resampled =====
============= grid 10200 resampled =====
============= grid 10203 resampled =====
============= grid 10204 resampled =====
============= grid 10205 resampled =====
============= grid 10216 resampled =====
============= grid 10217 resampled =====
============= gr

============= grid 10696 resampled =====
============= grid 10697 resampled =====
============= grid 10699 resampled =====
============= grid 10700 resampled =====
============= grid 10701 resampled =====
============= grid 10702 resampled =====
============= grid 10703 resampled =====
============= grid 10704 resampled =====
============= grid 10705 resampled =====
============= grid 10738 resampled =====
============= grid 10739 resampled =====
============= grid 10740 resampled =====
============= grid 10741 resampled =====
============= grid 10742 resampled =====
============= grid 10743 resampled =====
============= grid 10744 resampled =====
============= grid 10745 resampled =====
============= grid 10746 resampled =====
============= grid 10747 resampled =====
============= grid 10748 resampled =====
============= grid 10749 resampled =====
============= grid 10750 resampled =====
============= grid 10751 resampled =====
============= grid 10752 resampled =====
============= gr

## Validating the resampling :

In [5]:
final_df.shape

(55879484, 2)

In [7]:
len(df_groups)

6259

In [86]:
len(df_groups)

6259

In [6]:
final_df.index[-1]

Timestamp('2014-01-01 23:50:00')

In [113]:
final_df.index[-1]

Timestamp('2014-01-01 23:50:00')

In [9]:
unique_times = final_df.index.value_counts()

In [10]:
unique_times[unique_times != 6259]

2014-01-01 05:30:00    6258
2014-01-01 06:40:00    6258
2014-01-01 05:40:00    6258
2014-01-01 05:50:00    6258
2014-01-01 06:00:00    6258
                       ... 
2013-11-01 00:50:00    6246
2013-11-01 00:40:00    6246
2013-11-01 00:30:00    6246
2013-11-01 00:10:00    6245
2013-11-01 00:00:00    6245
Name: time, Length: 220, dtype: int64

In [112]:
unique_times[unique_times != 6259]

2014-01-01 05:30:00    6258
2014-01-01 06:40:00    6258
2014-01-01 05:40:00    6258
2014-01-01 05:50:00    6258
2014-01-01 06:00:00    6258
                       ... 
2013-11-01 00:50:00    6246
2013-11-01 00:40:00    6246
2013-11-01 00:30:00    6246
2013-11-01 00:10:00    6245
2013-11-01 00:00:00    6245
Name: time, Length: 220, dtype: int64

In [89]:
len(df.index)

6259

In [84]:
final_df.head()

,grid_square,internet_cdr
time,,
2013-11-01 00:00:00,38,11.517528
2013-11-01 00:10:00,38,10.134993
2013-11-01 00:20:00,38,9.689378
2013-11-01 00:30:00,38,9.821119
2013-11-01 00:40:00,38,9.047048


In [129]:
grids_list_groups

[38,
 39,
 40,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 271,
 272,
 273,
 274,
 275,
 276,
 277,
 278,
 285,
 286,
 288,
 289,
 388,
 389,
 390,
 391,
 392,
 393,
 394,
 395,
 397,
 401,
 402,
 403,
 404,
 405,
 406,
 501,
 502,
 503,
 504,
 505,
 506,
 507,
 508,
 509,
 510,
 511,
 512,
 513,
 514,
 515,
 516,
 517,
 518,
 519,
 520,
 521,
 522,
 523,
 617,
 618,
 621,
 622,
 623,
 624,
 625,
 626,
 627,
 628,
 629,
 630,
 631,
 632,
 633,
 634,
 635,
 636,
 637,
 638,
 639,
 640,
 641,
 734,
 735,
 736,
 738,
 739,
 740,
 741,
 742,
 743,
 744,
 745,
 746,
 747,
 748,
 749,
 750,
 751,
 752,
 753,
 754,
 755,
 756,
 757,
 758,
 759,
 851,
 852,
 853,
 856,
 857,
 858,
 859,
 860,
 861,
 862,
 863,
 864,
 865,
 866,
 867,
 868,
 869,
 870,
 871,
 872,
 873,
 874,
 875,
 876,
 877,
 969,
 970,
 971,
 972,
 974,
 975,
 976,
 977,
 978,
 979,
 980,
 981,
 982,
 983,
 984,
 985,
 986,
 987,
 988,
 989,
 990,
 991,
 992,
 993,
 994,
 1086,
 1087,
 1088,
 1089,
 1090,
 1091,
 1092,
 1093,

In [126]:
grids_list_groups = list(df.groups.keys())

# Converting the data to 2d snapshot for every time interval.

In [ ]:
time_steps = final_data.reset_index()['time'].unique()
grids = final_data.reset_index()['grid_square'].unique()

print("we have {} time steps, and {} grid sqaures".format(time_steps, grids))

In [ ]:
# =========== 2D Array ====================================
# Define the new data shape
time_steps = full_data.reset_index()['time'].unique()
grids = full_data.reset_index()['grid_square'].unique()

## Convert the pandas dataframe to a numpy array with shape()
# Each sample in the 4-D array is an image represent one snapshot taken at time-step
# to the city, each value represent the value of one grid at this time-step.
data = np.zeros(dtype=np.float64, shape=(len(time_steps), len(grids)//98, len(grids)//117, 1))

for i, j in enumerate(time_steps):
    data[i, :, :, :] = df.loc[j].to_numpy().reshape(98, 117, 1)

# save to npy file
np.save('full_grid.npy', data)